# Forte Tutorial 1.02: Forte's determinant class

---

In this tutorial we are going to explore how to create a simple FCI code using forte's Python API.

## Import modules
Here we will use modules to run an initial SCF computation and provide the integrals

In [1]:
import psi4
import forte

First we will use the module `ObjectsUtilPsi4` to perform a Hartree–Fock computation and prepare all the Forte objects. Here we freeze one orbital and indicated that two more should be treated as restricted (not included in the active space):

In [ ]:
# setup molecule object
molecule = psi4.geometry("""
O
H 1 1.0
H 1 1.0 2 180.0
""")

# Setup forte and prepare the active space integral class
mos_spaces = {'FROZEN_DOCC' :     [1,0,0,0,0,0,0,0], # freeze the O 1s orbital
              'RESTRICTED_DOCC' : [1,0,0,0,0,1,0,0]}

# Run the module ObjectsUtilPsi4 to generate the Forte data object
data = forte.modules.ObjectsUtilPsi4(molecule=molecule, basis="sto-3g", mo_spaces=mos_spaces).run()

Now we are ready to grab the objects that we will need: the active space integrals and the orbital information:

In [ ]:
as_ints = data.as_ints
mo_space_info = data.mo_space_info
psi_wfn = data.psi_wfn

print(f'Frozen-core energy = {as_ints.frozen_core_energy()}')
print(f'Nuclear repulsion energy = {as_ints.nuclear_repulsion_energy()}')
print(f'Scalar energy = {as_ints.scalar_energy()}')

## Creating determinants

Determinants are represented by the class `Determinant`. Here we create an empty determinant and print it by invoking the `str` function. This function prints the entire determinant (which has fixed size):

In [ ]:
d = forte.Determinant()
print(f'Determinant: {d}')

If we are working with only a few orbitals we can use the `.str()` function to produce a more compact representation:

In [ ]:
nact = mo_space_info.size('ACTIVE')
print(f'Determinant: {d.str(nact)}') # print only the active space part of the determinant (4 orbital)

There are several ways one can create determinants. The utility function `forte.det` converts a string to a determinant:

In [ ]:
d = forte.det("2+0-ab")
print(f'Determinant: {d}')

## Applying creation and annihilation operators to determinants

Determinants can be modified by applying second quantized creation and annihilation operators.
To apply the creation operator $\hat{a}^\dagger_1$, which adds one electron in the spin orbital $\phi_{i,\alpha}$, we can use the function (`create_alfa_bit`). This function returns the corresponding sign and modifies the original determinant (this is done for performance reasons):

In [ ]:
d = forte.Determinant()
print(f'Original determinant: {d.str(nact)}')

sign = d.create_alfa_bit(1)
print(f'New determinant:      {d.str(nact)}, sign = {sign}')

Here we create an electron in spin orbital 2 beta 

In [ ]:
sign = d.create_beta_bit(2)
print(f'Determinant: {d.str(nact)}, sign = {sign}')

Similarly, we can remove (annihilate) an electron with the command `destroy_alfa_bit` (`destroy_beta_bit` for the beta case). Since this orbital is empty, the sign returned is 0

In [ ]:
sign = d.destroy_alfa_bit(2)
print(f'Determinant: {d.str(nact)}, sign = {sign}')

## Creating the HF determinant

Next we do some bookeeping to find out the occupation of the Hartree-Fock determinant using the occupation returned to us by psi4

In [ ]:
# number of irreps
nirrep = mo_space_info.nirrep()

# number of doubly occupied orbitals per irrep
ndoccpi = mo_space_info.dimension('FROZEN_DOCC') + mo_space_info.dimension('RESTRICTED_DOCC')

# number of active orbitals per irrep
nactpi = mo_space_info.dimension('ACTIVE').to_tuple()

# compute the number of alpha electrons per irrep (total alpha - ndocc)
nact_aelpi = psi_wfn.nalphapi() - ndoccpi
nact_aelpi = nact_aelpi.to_tuple()      

# compute the number of beta electrons per irrep (total beta - ndocc)
nact_belpi = psi_wfn.nbetapi() - ndoccpi
nact_belpi = nact_belpi.to_tuple()

print(f'Number of alpha electrons per irrep: {nact_aelpi}')
print(f'Number of beta electrons per irrep:  {nact_belpi}')

ref = forte.Determinant()

# we loop over each irrep and fill the occupied orbitals 
irrep_start = [sum(nactpi[:h]) for h in range(nirrep)]
for h in range(nirrep):
    for i in range(nact_aelpi[h]): ref.set_alfa_bit(irrep_start[h] + i, True)
    for i in range(nact_belpi[h]): ref.set_beta_bit(irrep_start[h] + i, True)        
    
print(f'Reference determinant: {ref.str(nact)}')

We can now compute the energy of the determinant as $\langle \Phi | \hat{H} | \Phi \rangle$ using the `slater_rules` function in the `ActiveSpaceIntegrals` class

In [ ]:
as_ints.slater_rules(ref,ref) + as_ints.scalar_energy() + as_ints.nuclear_repulsion_energy() 

## Creating the FCI determinant basis

Forte provides the utility function `hilbert_space` that uses symmetry information to generate only those determinants that have the desired symmetry and given number of alpha/beta electrons. This function runs in C++ and will return a list of determinants:

In [ ]:
# active orbitals per irrep
symmetry = mo_space_info.symmetry('ACTIVE')
print(f'Symmetry of active space: {symmetry}')

nact_ael = sum(nact_aelpi)
nact_bel = sum(nact_belpi)

print(f'Number of alpha active electrons: {nact_ael}')
print(f'Number of beta active electrons:  {nact_bel}')

dets = forte.hilbert_space(nact,nact_ael,nact_bel,nirrep,symmetry)

print(f'==> List of FCI determinants <==')
for d in dets:
    print(f'{d.str(4)}')

## Diagonalize the Hamiltonian in the FCI space

In the last step, we diagonalize the Hamiltonian in the FCI determinant basis. We use the function `hamiltonian_matrix` to compute the matrix elements $\langle \Phi_I | \hat{H} | \Phi_J \rangle$.

In [ ]:
import numpy as np

# build the Hamiltonian matrix
H = forte.hamiltonian_matrix(dets,as_ints).to_array()

# diagonalize the Hamiltonian
evals, evecs = np.linalg.eigh(H)

psi4_fci = -74.846380133240530
print(f'FCI Energy = {evals[0] + as_ints.scalar_energy() + as_ints.nuclear_repulsion_energy()}')
print(f'FCI Energy Error = {evals[0] + as_ints.scalar_energy() + as_ints.nuclear_repulsion_energy()- psi4_fci}')

index_hf = dets.index(ref)
print(f'Index of the HF determinant in the FCI vector {index_hf}')